In [251]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os
from PIL import Image
import random
from tqdm import tqdm
import cv2
import torch
from torch import optim, nn
from torchvision import models, transforms
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.cluster import KMeans, DBSCAN
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from time import time
from sklearn.pipeline import make_pipeline
from sklearn.dummy import DummyClassifier

In [37]:
# - also create labels to be int from 0 to 1 for each class and somehow randomize the data and then split it between data and labels 
def load_data(base_path: Path, class_names: list, train: bool, resolution: tuple, randomized: bool, flag: str = "RGB"):
    data = []
    labels = []
    class_maping = {k:i for i,k in enumerate(class_names)}
    num_images_per_class = 1000 if train else 150
    train = "train" if train else "test"
    print(f"Loading each class for {train}.")
    for class_name in class_names:
        class_folder_path = base_path/train/class_name
        for image_name in tqdm(os.listdir(class_folder_path)[0:num_images_per_class]):
            
            img = np.array(Image.open(class_folder_path/image_name).convert(flag).resize(resolution))

            img_label = class_maping[class_name]
            if img is None:
                    print(f'This image is bad: {class_folder_path/image_name}')
            else:
                labels.append(img_label)
                data.append(img)
    
    if randomized:
        randomized_indices = np.random.permutation(len(labels))
        data, labels = np.array(data)[randomized_indices], np.array(labels)[randomized_indices]
    
    return np.array(data), np.array(labels)

In [38]:
base_path = Path("Z:/Master I/PML - Practical Machine Learning/Unsupervised_Comparison/data/Architectural_Heritage_Elements")

class_names = ['altar', 'apse', 'bell_tower', 'column','dome(inner)','dome(outer)',
               'flying_buttress','gargoyle','stained_glass','vault']

# class_names = ['altar', 'column','dome(outer)','gargoyle','stained_glass']

train_data, train_labels = load_data(base_path, class_names, train = True, resolution=(64,64), randomized = True, flag = 'RGB')
# display_some_images(train_data, train_labels)

test_data, test_labels = load_data(base_path, class_names, train = False, resolution = (64,64), randomized = True, flag = 'RGB')
# display_some_images(test_data, test_labels)


Loading each class for train.


100%|██████████| 1000/1000 [00:03<00:00, 264.04it/s]


Loading each class for test.


100%|██████████| 150/150 [00:00<00:00, 520.40it/s]


In [8]:
print(test_data.shape)
print(test_labels.shape)
print(train_data.shape)
print(train_labels.shape)

# train_data = train_data.reshape(6399,28,28,1)
# test_data = test_data.reshape(1221,28,28,1)

(1221, 64, 64, 3)
(1221,)
(6399, 64, 64, 3)
(6399,)


In [40]:
def normalize_and_flatten(data: np.array):
    normalized = data / 255.0
    # mean = np.mean(arr, axis = 0)
    preproc_data = normalized.reshape(len(normalized), -1 )
    return preproc_data

In [41]:
test_features = normalize_and_flatten(test_data)
train_features = normalize_and_flatten(train_data)

In [35]:
class FeatureExtractor(nn.Module):
  def __init__(self, model):
    super(FeatureExtractor, self).__init__()

    self.features = list(model.features)
    self.features = nn.Sequential(*self.features)
    self.pooling = model.avgpool
    self.flatten = nn.Flatten()
    self.fc = model.classifier[0]
  
  def forward(self, x):
    out = self.features(x)
    out = self.pooling(out)
    out = self.flatten(out)
    out = self.fc(out) 
    return out 

In [34]:
def extract_features(data, _transform, device):
    model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
    new_model = FeatureExtractor(model)
    new_model = new_model.to(device)
    features = []
    for i in tqdm(range(len(data))):
        transformed_img = _transform(data[i])
        img = transformed_img.reshape(1, 3, 64, 64)
        img = img.to(device)
        with torch.no_grad():
            feature = new_model(img)
        features.append(feature.cpu().detach().numpy().reshape(-1))

    return np.array(features)

In [41]:
transform = transforms.Compose([
  transforms.ToPILImage(),
  transforms.Resize(64),
  transforms.ToTensor() # this normalizes the data too                       
])


model = models.vgg16(pretrained=True)
new_model = FeatureExtractor(model)


device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
new_model = new_model.to(device)

train_features = extract_features(train_data, transform, device)
test_features = extract_features(test_data, transform, device)

z:\anaconda3\envs\PML_Project2\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
z:\anaconda3\envs\PML_Project2\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 1221/1221 [00:17<00:00, 70.78it/s]


In [51]:
print(test_data.shape)
print(test_labels.shape)
test_features = np.array(test_features)
print(test_features.shape)


print(train_data.shape)
print(train_labels.shape)
train_features = np.array(train_features)
print(train_features.shape)

(1221, 64, 64, 3)
(1221,)
(1221, 4096)
(8325, 64, 64, 3)
(8325,)
(8325, 4096)


In [31]:
np.save("./data/saved_features/train_features_64_64.npy",train_features)
np.save("./data/saved_features/test_features_64_64.npy",test_features)

In [11]:
def get_labels_mapping(cluster_labels,y_train):
    """
    Associates most probable label with each cluster in KMeans model
    returns: dictionary of clusters assigned to each label
    """
    labels_mapping = {}
    nr_of_clusters = len(np.unique(cluster_labels))
    for i in range(nr_of_clusters):
        indexes = np.array([1 if cluster_labels[ind]==i else 0 for ind in range(len(cluster_labels))]) # lista noua cu 1 daca valoarea din cluster e egala cu valoarea lui i si 0 altfel 
        num = np.bincount(y_train[indexes==1]).argmax()
        labels_mapping[i] = num

    return labels_mapping

# K-means

In [98]:
print(80 * "_")
print("clusters\tinertia\t\thomo\tsilhouette\ttrain_acc\ttest_acc")
for i in [10,15,20,25,50,80,160]:
    # print("Number of clusters: {}".format(i))
    kmeans = KMeans(init="k-means++", n_clusters=i, n_init=4)
    kmeans.fit(train_features)

    _metrics = [i,
                kmeans.inertia_,
                metrics.homogeneity_score(train_labels,kmeans.labels_),
                metrics.silhouette_score(train_features,kmeans.labels_,metric="euclidean")]

    labels_mapping = get_labels_mapping(kmeans.labels_,train_labels)


    number_labels_train = np.zeros(len(kmeans.labels_))
    for i in range(len(kmeans.labels_)):
        number_labels_train[i] = labels_mapping[kmeans.labels_[i]]
    _metrics.append(accuracy_score(number_labels_train,train_labels))


    predicted = kmeans.predict(test_features)
    number_labels_test = np.zeros(len(predicted))
    for i in range(len(predicted)):
        number_labels_test[i] = labels_mapping[predicted[i]]
    _metrics.append(accuracy_score(number_labels_test,test_labels))

    formatter_result = ("{}\t\t{:.3f}\t{:.3f}\t{:.3f}\t\t{:.3f}\t\t{:.3f}")

    print(formatter_result.format(*_metrics))
print(80 * "_")

________________________________________________________________________________
clusters	inertia		homo	silhouette	train_acc	test_acc
10		272721472.000	0.368	0.055		0.533		0.528
15		254222112.000	0.438	0.045		0.576		0.541
20		242452624.000	0.460	0.049		0.566		0.532
25		233697168.000	0.485	0.041		0.594		0.565
50		209469504.000	0.543	0.028		0.648		0.609
80		195258304.000	0.581	0.030		0.678		0.622
160		174850864.000	0.621	0.027		0.699		0.645
________________________________________________________________________________


In [73]:
def tune_k_means(kmeans, name, data, labels):

    starting_time = time()
    estimator = make_pipeline(StandardScaler(), kmeans).fit(data)
    fit_time = time() - starting_time

    _metrics = [name,
                fit_time,
                estimator[-1].inertia_,
                metrics.homogeneity_score(labels, estimator[-1].labels_),
                metrics.silhouette_score(data,
                                         estimator[-1].labels_,
                                         metric="euclidean",
                                         sample_size=300,)]

    # Print the results
    formatter_result = ("{:9s}\t{:.3f}s\t{:.3f}\t{:.3f}\t{:.3f}")
    print(formatter_result.format(*_metrics))

In [74]:
print(60 * "_")
print("init\t\ttime\tinertia\t\thomo\tsilhouette")

kmeans = KMeans(init="k-means++", n_clusters=25, n_init=4, random_state=0)
tune_k_means(kmeans=kmeans, name="k-means++", data=train_features, labels=train_labels)

kmeans = KMeans(init="random", n_clusters=25, n_init=4, random_state=0)
tune_k_means(kmeans=kmeans, name="random", data=train_features, labels=train_labels)

pca = PCA(n_components=25).fit(train_features)
kmeans = KMeans(init=pca.components_, n_clusters=25, n_init=1)
tune_k_means(kmeans=kmeans, name="PCA-based", data=train_features, labels=train_labels)

print(60  * "_")

____________________________________________________________
init		time	inertia		homo	silhouette
k-means++	22.698s	20177540.000	0.486	0.046
random   	7.690s	20164680.000	0.487	0.028
PCA-based	2.560s	20156484.000	0.492	0.032
____________________________________________________________


In [99]:
def tune_k_means(kmeans, name, data, labels):

    starting_time = time()
    estimator =  kmeans.fit(data)
    fit_time = time() - starting_time

    _metrics = [name,
                fit_time,
                kmeans.inertia_,
                metrics.homogeneity_score(labels, kmeans.labels_),
                metrics.silhouette_score(data,
                                         kmeans.labels_,
                                         metric="euclidean",
                                         sample_size=300,)]

    labels_mapping = get_labels_mapping(kmeans.labels_,train_labels)


    number_labels_train = np.zeros(len(kmeans.labels_))
    for i in range(len(kmeans.labels_)):
        number_labels_train[i] = labels_mapping[kmeans.labels_[i]]
    _metrics.append(accuracy_score(number_labels_train,train_labels))

    predicted = kmeans.predict(test_features)
    number_labels_test = np.zeros(len(predicted))
    for i in range(len(predicted)):
        number_labels_test[i] = labels_mapping[predicted[i]]
    _metrics.append(accuracy_score(number_labels_test,test_labels))

    formatter_result = ("{:.1}\t\t{:.3f}\t{:.3f}\t{:.3f}\t\t{:.3f}\t\t{:.3f}")
    
    # Print the results
    formatter_result = ("{:9s}\t{:.3f}s\t{:.3f}\t{:.3f}\t{:.3f}\t\t{:.3f}\t\t{:.3f}")
    print(formatter_result.format(*_metrics))

In [102]:
print(90 * "_")
print("algorithm\ttime\tinertia\t\thomo\tsilhouette\tacc_train\tacc_test")

kmeans = KMeans(init="k-means++", n_clusters=25, n_init=4, random_state=0, algorithm="lloyd")
tune_k_means(kmeans=kmeans, name="lloyd", data=train_features, labels=train_labels)

kmeans = KMeans(init="k-means++", n_clusters=25, n_init=4, random_state=0,algorithm="elkan")
tune_k_means(kmeans=kmeans, name="elkan", data=train_features, labels=train_labels)


print(90 * "_")

__________________________________________________________________________________________
algorithm	time	inertia		homo	silhouette	acc_train	acc_test
lloyd    	24.657s	233758128.000	0.481	0.034		0.587		0.545
elkan    	21.489s	233758144.000	0.481	0.026		0.587		0.545
__________________________________________________________________________________________


In [ ]:
# not working 
parameters = {'init':['k-means++', 'random'], 'n_init':["auto", 4 , 10]}

kmeans_pipe = Pipeline([('knn', KMeans(n_clusters=25))])

def silhouette_score(estimator, X):
    clusters = estimator.fit_predict(X)
    score = metrics.silhouette_score(train_features, clusters, metric='precomputed')
    return score

N = len(train_features)
cv_custom = [(range(0,N))]

clf = GridSearchCV(kmeans_pipe,
                   parameters,
                   verbose=3,
                   scoring=silhouette_score,
                   cv=cv_custom
                   )

clf.fit(train_features)

# DBscan

In [231]:
def tune_db_scan(dbscan, value, data, labels):

    starting_time = time()
    estimator = make_pipeline(StandardScaler(), dbscan).fit(data)
    fit_time = time() - starting_time

    # core_sample_mask = np.zeros_like(estimator[-1].labels_,dtype=bool)
    # core_sample_mask[estimator[-1].core_sample_indices] = True

    _metrics = [value,
                fit_time,
                metrics.silhouette_score(data,
                                         estimator[-1].labels_),
                metrics.homogeneity_score(labels, estimator[-1].labels_),
                metrics.completeness_score(labels, estimator[-1].labels_),
                len(np.unique(dbscan.labels_))]

    # Print the results
    formatter_result = ("{}\t\t{:.3f}\t{:.3f}\t\t{:.3f}\t{:.3f}\t{}")
    print(formatter_result.format(*_metrics))

In [250]:
print(80 * "_")
print("eps_val\t\ttime\tsilhouette\thomo\tcomplet\tnr_clusters")

for i in [10,20,30,40,60,80]:
    db = DBSCAN(eps=i, min_samples=3)
    tune_db_scan(dbscan=db, value=i, data=train_features, labels=train_labels)

print(80 * "_")


________________________________________________________________________________
eps_val		time	silhouette	homo	complet	nr_clusters
10		8.420	-0.014		0.000	0.238	2
20		7.597	-0.201		0.005	0.252	12
30		7.603	-0.307		0.034	0.140	45
40		7.354	-0.135		0.042	0.118	27
60		7.871	0.264		0.011	0.112	5
80		9.779	0.395		0.001	0.097	3
________________________________________________________________________________


In [249]:
print(80 * "_")
print("min_samples\ttime\tsilhouette\thomo\tcomplet\tnr_clusters")

for i in [2,3,4,5,6,7]:
    db = DBSCAN(eps=40, min_samples=i)
    tune_db_scan(dbscan=db, value=i, data=train_features, labels=train_labels)

print(80 * "_")

________________________________________________________________________________
min_samples	time	silhouette	homo	complet	nr_clusters
2		7.815	-0.267		0.068	0.148	143
3		7.899	-0.135		0.042	0.118	27
4		7.125	-0.057		0.035	0.106	11
5		7.620	0.034		0.031	0.098	6
6		7.477	0.023		0.032	0.100	6
7		7.844	0.053		0.031	0.098	4
________________________________________________________________________________


In [248]:
print(80 * "_")
print("min_samples\ttime\tsilhouette\thomo\tcomplet\tnr_clusters")

for i in [2,3,4,5,6,7]:
    db = DBSCAN(eps=60, min_samples=i)
    tune_db_scan(dbscan=db, value=i, data=train_features, labels=train_labels)

print(80 * "_")

________________________________________________________________________________
min_samples	time	silhouette	homo	complet	nr_clusters
2		7.994	0.149		0.014	0.125	19
3		7.874	0.264		0.011	0.112	5
4		7.518	0.098		0.012	0.117	4
5		7.673	0.112		0.012	0.119	3
6		8.252	0.291		0.012	0.119	2
7		7.846	0.291		0.013	0.125	2
________________________________________________________________________________


In [243]:
print(80 * "_")
print("min_samples\ttime\tsilhouette\thomo\tcomplet\tnr_clusters")

for i in [2,3,4,5,6,7]:
    db = DBSCAN(eps=80, min_samples=i)
    tune_db_scan(dbscan=db, value=i, data=train_features, labels=train_labels)

print(80 * "_")

________________________________________________________________________________
min_samples	time	silhouette	homo	complet	nr_clusters
2		8.929	0.368		0.002	0.110	5
3		8.719	0.395		0.001	0.097	3
4		8.641	0.409		0.001	0.087	2
5		8.845	0.409		0.001	0.087	2
6		9.110	0.409		0.001	0.087	2
7		9.075	0.409		0.001	0.087	2
________________________________________________________________________________


# Comparison random, supervisez, dbscan, kmeans

In [252]:
# sanity check
svm = SVC(kernel= 'linear')
svm.fit(train_features, train_labels)
y_pred = svm.predict(test_features)
print(accuracy_score(test_labels, y_pred))

0.8247338247338247


In [253]:
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(train_features, train_labels)
y_pred = dummy_clf.predict(test_features)
print(accuracy_score(test_labels, y_pred))

0.12285012285012285


In [303]:
def results(algo):


    if algo == "random":
        starting_time = time()
        dummy_clf = DummyClassifier(strategy="most_frequent")
        dummy_clf.fit(train_features, train_labels)
        fit_time = time() - starting_time
        y_pred_test = dummy_clf.predict(test_features)
        y_pred_train = dummy_clf.predict(train_features)

        _metrics = [algo,
                    fit_time,
                    "none",
                    accuracy_score(train_labels, y_pred_train),
                    accuracy_score(test_labels, y_pred_test)]
    elif algo == "SVM":
        starting_time = time()
        svm = SVC(kernel= 'linear')
        svm.fit(train_features, train_labels)
        fit_time = time() - starting_time
        y_pred_test = svm.predict(test_features)
        y_pred_train = svm.predict(train_features)
        _metrics = [algo,
                    fit_time,
                    "none",
                    accuracy_score(train_labels, y_pred_train),
                    accuracy_score(test_labels, y_pred_test)]
    elif algo == "kmeans":
        starting_time = time()
        kmeans = KMeans(init="k-means++", n_clusters=25, n_init=4)
        kmeans.fit(train_features)
        fit_time = time() - starting_time
        labels_mapping = get_labels_mapping(kmeans.labels_,train_labels)

        number_labels_train = np.zeros(len(kmeans.labels_))
        for i in range(len(kmeans.labels_)):
            number_labels_train[i] = labels_mapping[kmeans.labels_[i]]

        predicted = kmeans.predict(test_features)
        number_labels_test = np.zeros(len(predicted))
        for i in range(len(predicted)):
            number_labels_test[i] = labels_mapping[predicted[i]]

        _metrics =[algo,
                  fit_time,
                  metrics.silhouette_score(train_features,kmeans.labels_,metric="euclidean"),
                  accuracy_score(number_labels_train,train_labels),
                  accuracy_score(number_labels_test,test_labels)]

    elif algo == "dbscan":
        starting_time = time()
        estimator = make_pipeline(StandardScaler(), DBSCAN(eps=80, min_samples=5)).fit(train_features)
        fit_time = time() - starting_time

        _metrics = [algo,
                    fit_time,
                    metrics.silhouette_score(train_features,
                                             estimator[-1].labels_),
                    "none",
                    "none"]

    formatter_result = ("{}\t\t{}\t{}\t\t\t{}\t\t{}")
    print(formatter_result.format(*_metrics))

In [298]:
print(100 * "_")
print("algorithm\ttime\t\t\tsilhouette\t\t\tacc_train\t\t\tacc_test")

results("random")
results("SVM")
results("kmeans")
results("dbscan")
print(100 * "_")

____________________________________________________________________________________________________
algorithm	time			silhouette		acc_train		acc_test
random		0.0010023117065429688	none		0.12012012012012012	0.12285012285012285
SVM		34.203367948532104	none		0.9997597597597597	0.8247338247338247
kmeans		31.14898657798767	0.04792613908648491		0.6263063063063063	0.5831285831285832
dbscan		7.996812105178833	-0.20988327264785767		none	none
____________________________________________________________________________________________________


In [304]:
print(120 * "_")
print("algorithm\ttime\t\t\tsilhouette\t\t\tacc_train\t\t\tacc_test")

results("random")
results("SVM")
results("kmeans")
results("dbscan")
print(120 * "_")

________________________________________________________________________________________________________________________
algorithm	time			silhouette			acc_train			acc_test
random		0.002003908157348633	none			0.12012012012012012		0.12285012285012285
SVM		35.261393785476685	none			0.9997597597597597		0.8247338247338247
kmeans		26.77710747718811	0.042468223720788956			0.6072072072072072		0.579033579033579
dbscan		23.04601550102234	0.40913838148117065			none		none
________________________________________________________________________________________________________________________
